In [ ]:
kernel.silent(true)

In [ ]:
import coursierapi.MavenRepository
interp.repositories() ++= Seq(MavenRepository.of("https://jitpack.io"))

In [ ]:
import $ivy.`com.github.propi:rdfrules:1.5.0`
import collection._
import org.apache.jena.riot.Lang

import com.github.propi.rdfrules.data._
import com.github.propi.rdfrules.algorithm.amie._
import com.github.propi.rdfrules.algorithm.dbscan._
import com.github.propi.rdfrules.utils._
import com.github.propi.rdfrules.index._
import com.github.propi.rdfrules.rule._
import com.github.propi.rdfrules.ruleset._

In [ ]:
val graph = Graph("data/czso-average-salaries.trig")
graph.cache("graph.cache")

In [ ]:
val graph = Graph.fromCache("graph.cache")

In [ ]:
var dataset = Dataset() + graph
dataset.cache("dataset.cache")

In [ ]:
var dataset = Dataset.fromCache("dataset.cache")

In [ ]:
val types: Map[TripleItem.Uri, Map[TripleItemType, Int]] = dataset.types()

In [ ]:
dataset.size

In [ ]:
val histogram: Map[Histogram.Key, Int] = dataset.histogram(predicate = true, `object` = true)

In [ ]:
val index = dataset.index()
index.cache("index.cache")

In [ ]:
import eu.easyminer.discretization.impl.Interval
dataset = dataset.discretize(DiscretizationTask.Equifrequency(5))(quad => quad.triple.predicate.hasSameUriAs("http://data.czso.cz/ontology/medianMzdy"))
dataset = dataset.discretize(DiscretizationTask.Equifrequency(5))(quad => quad.triple.predicate.hasSameUriAs("http://data.czso.cz/ontology/prumernaMzda"))

In [ ]:
val index = Index.fromCache("index.cache", false)

In [ ]:
Debugger() { implicit debugger =>
    val miningTask = Amie()
    index.mine(miningTask)
}

In [ ]:
com.github.propi.rdfrules.rule.AtomPattern.AtomItemPattern
AtomPattern.AtomItemPattern.Constant
new AtomPattern.AtomItemPattern.Constant(null)
AnyConstant
ConstantsPosition.Object

In [ ]:
ConstantsPosition.Object

In [ ]:
val constantsAtObject = RuleConstraint.ConstantsAtPosition.ConstantsPosition.Object
val onlyObjectConstants = RuleConstraint.ConstantsAtPosition(constantsAtObject)
val constantsNowhere = RuleConstraint.ConstantsAtPosition.ConstantsPosition.Nowhere
val noConstants = RuleConstraint.ConstantsAtPosition(constantsNowhere)

val preparedMiningTask = Amie()
.addThreshold(Threshold.MinHeadCoverage(0.2))
.addThreshold(Threshold.MinSupport(20))
.addConstraint(onlyObjectConstants)
.addPattern(
AtomPattern(
    predicate = TripleItem.Uri("http://data.czso.cz/ontology/medianMzdy"), `object`=AnyConstant)
)

preparedMiningTask
/*.addConstraint(onlyObjectConstants)
.addPattern(
        AtomPattern(predicate = TripleItem.Uri("isMarriedTo"))  // * => isMarriedTo(Any, Any)
)
.addPattern(
        AtomPattern(subject = 'a', predicate = TripleItem.Uri("isCitizenOf"), `object` = 'b') 
    &:  AtomPattern(subject = 'b', predicate = TripleItem.Uri("participatedIn")) 
    //=>: None
    =>: AtomPattern(subject = 'a')
)*/


In [ ]:
var ruleset = index.mine(preparedMiningTask)
ruleset.cache("rules.cache")
ruleset.foreach(println)
preparedMiningTask

In [ ]:
var ruleset = Ruleset.fromCache("rules.cache")

In [ ]:
/*var ruleset = index.mine(preparedMiningTask)
ruleset = ruleset
.filter(rule => rule.measures(Measure.HeadCoverage).value > 0.02)
.computeConfidence(0.5)
.computePcaConfidence(0.5)
.computeLift()
.sortBy(Measure.HeadCoverage, Measure.Lift)
.makeClusters(DbScan(minNeighbours = 3, minSimilarity = 0.85))
    
ruleset.export("rules.txt")
println(ruleset.size)
*/
ruleset.foreach(println)